In [1]:
import os
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Current working directory of the notebook
notebook_directory = os.getcwd()

# Relative path to the dataset from the notebook directory
relative_path = r'..\data\raw\Cleaned_Data.csv'

# Construct the absolute path to the dataset
dataset_path = os.path.normpath(os.path.join(notebook_directory, relative_path))

In [3]:
df = pd.read_csv(dataset_path)

In [4]:
df.drop(columns='Unnamed: 0', axis=1, inplace=True)

In [5]:
df.head()

,year,month,palestinians_injuries,israelis_injuries,palestinians_killed,israelis_killed
0,2000,DECEMBER,781,0,51,8
1,2000,NOVEMBER,3838,0,112,22
2,2000,OCTOBER,5984,0,104,10
3,2000,SEPTEMBER,0,0,16,1
4,2001,DECEMBER,304,0,67,36


In [6]:
# Group the data by year and calculate the total injuries and deaths for Palestinians and Israelis
grouped_data = df.groupby('year').sum()[['palestinians_injuries', 'israelis_injuries', 'palestinians_killed', 'israelis_killed']]
grouped_data.reset_index(inplace=True)  # Resetting index for Plotly

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div(style={'font-family': 'Arial, sans-serif', 'padding': '20px'}, children=[
    html.H1("Israeli-Palestinian Conflict Dashboard", style={'text-align': 'center'}),
    
    # Dropdown for selecting metric
    html.Div([
        html.Label("Select Metric:", style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='metric-dropdown',
            options=[
                {'label': 'Injuries', 'value': 'injuries'},
                {'label': 'Deaths', 'value': 'deaths'}
            ],
            value='injuries',
            style={'width': '100%', 'border-radius': '15px', 'margin-bottom': '20px'},
            className='box-shadow dropdown'
        ),
    ]),
    
    # Checkboxes for selecting groups
    html.Div([
        html.Label("Select Groups:", style={'font-weight': 'bold'}),
        dcc.Checklist(
            id='group-checkboxes',
            options=[
                {'label': 'Palestinians', 'value': 'palestinians'},
                {'label': 'Israelis', 'value': 'israelis'}
            ],
            value=['palestinians', 'israelis'],
            style={'width': '96%', 'margin-bottom': '20px', 'border-radius': '15px'},
            className='box-shadow'
        ),
    ]),
    
    # Create a Plotly Express line plot using dcc.Graph
    dcc.Graph(
        id='line-plot',
        style={'margin-bottom': '30px', 'box-shadow': '0px 0px 10px 0px #888888', 'padding': '15px', 'border-radius': '15px', 'width': '98%'}
    ),
    
    # Create a Plotly Express bar plot using dcc.Graph
    dcc.Graph(
        id='bar-plot',
        figure=px.bar(df[['palestinians_killed', 'year']], x='year', y='palestinians_killed',
                    color='palestinians_killed', animation_frame="year",
                    animation_group="palestinians_killed", log_x=True,
                    range_x=[1999, 2022], range_y=[0, 2500],
                    labels={'palestinians_killed': 'Number of Palestinians Killed'},
                    title='Number of Palestinians Killed Over Time'),
        style={'margin-bottom': '30px', 'box-shadow': '0px 0px 10px 0px #888888', 'padding': '15px', 'border-radius': '15px', 'width': '98%'}
    ),
    
    # Create a Plotly Express bar plot with interactivity using dcc.Graph
    dcc.Graph(
        id='monthly-killing-plot',
        figure=px.bar(data_frame=df, x='month', y=['palestinians_killed', 'israelis_killed'],
                    title='Monthly Killings',
                    labels={'value': 'Count', 'variable': 'Type', 'month': 'Month'},
                    barmode='group'),
        style={'margin-bottom': '30px', 'box-shadow': '0px 0px 10px 0px #888888', 'padding': '15px', 'border-radius': '15px', 'width': '98%'}
    )
])

# Define callback to update the line plot based on dropdown and checkboxes
@app.callback(
    Output('line-plot', 'figure'),
    [Input('metric-dropdown', 'value'),
    Input('group-checkboxes', 'value')]
)
def update_line_plot(selected_metric, selected_groups):
    # Adjust column names based on the selected metric
    metric_suffix = 'injuries' if selected_metric == 'injuries' else 'killed'
    
    # Filter data based on selected groups
    filtered_data = grouped_data[['year'] + [f'{group}_{metric_suffix}' for group in selected_groups]]
    
    # Rename columns for better readability
    columns_mapping = {f'{group}_{metric_suffix}': f'{group.capitalize()} {selected_metric.capitalize()}' for group in selected_groups}
    filtered_data.rename(columns=columns_mapping, inplace=True)
    
    # Plotting the data using Plotly Express
    fig = px.line(filtered_data, x='year',
                y=[f'{group.capitalize()} {selected_metric.capitalize()}' for group in selected_groups],
                labels={'value': f'Number of {selected_metric.capitalize()}', 'variable': 'Group'},
                title=f'{selected_metric.capitalize()} in Israel-Palestine Conflict')
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run(jupyter_mode='external')


Dash app running on http://127.0.0.1:8050/


C:\Users\Dell\AppData\Local\Temp\ipykernel_15556\2629359264.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dell\AppData\Local\Temp\ipykernel_15556\2629359264.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dell\AppData\Local\Temp\ipykernel_15556\2629359264.py:86: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Dell\AppData\Local\Temp\ipykernel_15556\2629359264.py:86: SettingWithCopyWarning:


A value